# Boxes Processing

Now that have computed the object detection on two sample images, we will display the results, and process them if needed!

## I. Visualization

### I.1. Loading

First things first: load the images and the dictionaries saved in pickle format at the end of the last challenge.

In [4]:
import tensorflow.compat.v1 as tf
import pickle

In [3]:
### TODO: Load the pickle dicts and the images
image1 = tf.keras.preprocessing.image.load_img('../../01-Boxes-Computation/input/image1.jpg')
image2 = tf.keras.preprocessing.image.load_img('../../01-Boxes-Computation/input/image2.jpg')
image3 = tf.keras.preprocessing.image.load_img('../../01-Boxes-Computation/input/image3.jpg')

In [8]:
output1, output2, output3 = pickle.load(open('../../01-Boxes-Computation/output/output_dicts.pkl', 'rb'))

### I.2. Playing with the labels

To make a proper visualization, we may want the labels as strings, not as integers. For that, we will need to use the label list, which is in `models/research/object_detection/data/mscoco_label_map.pbtxt`, in the folder you cloned in previous challenge. Save the path to this file into the variable `PATH_TO_LABELS`.

Using this path, as well as the functions `load_labelmap`, `convert_label_map_to_categories` and `create_category_index` from the `label_map_util` module, you can get all we will need for visualization!

In [31]:
PATH_TO_LABELS = '/home/guillaume/code/GGIML/models/research/object_detection/data/mscoco_label_map.pbtxt'

In [55]:
import os
os.environ['PYTHONPATH'] += '/home/guillaume/code/GGIML/models/research/:/home/guillaume/code/GGIML/models/research/slim' 

KeyError: 'PYTHONPATH'

In [56]:
os.environ

environ{'SHELL': '/usr/bin/zsh',
        'LSCOLORS': 'Gxfxcxdxbxegedabagacad',
        'SESSION_MANAGER': 'local/Swift-Guillaume:@/tmp/.ICE-unix/1436,unix/Swift-Guillaume:/tmp/.ICE-unix/1436',
        'PYENV_HOOK_PATH': '/home/guillaume/.pyenv/pyenv.d:/usr/local/etc/pyenv.d:/etc/pyenv.d:/usr/lib/pyenv/hooks:/home/guillaume/.pyenv/plugins/pyenv-virtualenv/etc/pyenv.d',
        'QT_ACCESSIBILITY': '1',
        'COLORTERM': 'truecolor',
        'PYENV_SHELL': 'zsh',
        'XDG_CONFIG_DIRS': '/etc/xdg/xdg-ubuntu:/etc/xdg',
        'LESS': '-R',
        'PYENV_ACTIVATE_SHELL': '1',
        'XDG_MENU_PREFIX': 'gnome-',
        'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated',
        'GTK_IM_MODULE': 'ibus',
        'QT4_IM_MODULE': 'ibus',
        'MANDATORY_PATH': '/usr/share/gconf/ubuntu.mandatory.path',
        'PYSPARK_DRIVER_PYTHON': 'jupyter',
        'GNOME_SHELL_SESSION_MODE': 'ubuntu',
        'SSH_AUTH_SOCK': '/run/user/1000/keyring/ssh',
        'PYSPARK_DRIVER_PYTHON_OPTS': 'n

In [50]:
import sys
sys.path.append('/home/guillaume/code/GGIML/models/research/')
from object_detection.utils import label_map_util

TypeError: Couldn't build proto file into descriptor pool!
Invalid proto descriptor for file "object_detection/protos/string_int_label_map.proto":
  object_detection/protos/string_int_label_map.proto: A file with this name is already in the pool.


In [57]:
# Here we use those functions for you
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

NameError: name 'string_int_label_map_pb2' is not defined

Play with the created variables and understand what they are.

### 1.3. Visualization of the object detection

Finally, use the function `display_image_with_boxes` provided below, display the images with the corresponding predictions. For each image, print the number of detected objects in the image.

In [48]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def tf_box_to_matplotlib(box, im):
    x = box[1]*im.shape[1]
    y = box[0]*im.shape[0]
    width = (box[3]-box[1])*im.shape[1]
    height = (box[2]-box[0])*im.shape[0]
    
    return x, y, width, height

def display_image_with_boxes(image, dico, threshold=0):

    fig, ax = plt.subplots(1, figsize=(14,10))
    plt.imshow(image)

    for i in range(dico['num_detections']):
        score = dico['detection_scores'][i]
        clas = category_index[dico['detection_classes'][i]]['name']

        if score >= threshold:

            display = str("{:.2f}".format(score))+' '+clas

            x, y, w, h = tf_box_to_matplotlib(dico['detection_boxes'][i], image)
            ax.add_patch(Rectangle((x, y), w, h,linewidth=1,edgecolor='r',facecolor='none'))
            plt.text(x, y-10, display, color='red', fontsize=10)

    plt.axis('off')
    plt.show()

In [ ]:
### TODO: use display_image_with_boxes and display the result

**What do you think?** Is it working good enough? In particular, compare the number of detected object and the number of objects you see in the image.

What could we do to improve our results?

## II. Post processing

We have to perform non max suppression in order to be really consistent. The first step is to compute the Intersection over Union (IoU).

### II.1. IoU

You will implement a function `iou(box1, box2)`, knowing that boxes in the dictionary follow the format `box = [ymin, xmin, ymax, xmax]`.

Computing the IoU may seem easy, but is actually a bit tricky: there are a lot of cases to take into account.

To help you out, consider the following Boxes A and B

<p align="center">
<img src="https://drive.google.com/uc?export=view&id=1m1ZmLEdNkMHY3DNiBrCUlOUUO_tlD8zQ">
</p>

The first step is to compute the **intersection** of those two boxes. Implement a function `intersection(boxA, boxB)` that computes the intersection of the boxes A and B.

There are 4 distinct cases to take into account:
- the blue box is on the top left of the red one
- the blue box is on the top right of the red one
- the blue box is on the bottom left of the red one
- the blue box is on the bottom right of the red one

Finally, there is one extreme case: when the two boxes have no intersection.

Feel free to take a sheet of paper and try out all those cases to implement the `intersect` function.

In [ ]:
### TODO: implement intersect(box1, box2)


Then you can compute the **union** of those two boxes. One can notice that the union is nothing but area of the two boxes, minus the intersection:
$$
union(A, B) = area(A) + area(B) - intersection(A, B)
$$

Now implement the function `union(boxA, boxB)`.

In [ ]:
### TODO: implement union(box1, box2)


Finally, implement the function `iou(box1, box2)` using the two functions you just implemented.

In [ ]:
### TODO: implement iou(box1, box2)


To make sure it works, feel free to test your IoU function on several examples.

### II.2. Non max suppression

Now comes the big part: you will have to implement the `non_max_suppression` function.

This function will take as input a dictionary (output of TensorFlow computed in previous challenge), a iou threshold and a detection threshold. This function should output a dictionary containing only the non supressed boxes and detections.

An easier way would be to make two functions. First, implement a function `apply_detection_threshold(detections, detection_threshold)` that removes all the boxes below a given `detection_threshold`.

Then, implement a function `non_max_suppression(detections, iou_threshold, detection_threshold)`. This function applies the detection threshold first with `detection_threshold`. Then, loop over all the boxes, keep the highest score, and remove all the boxes with an IoU below the `iou_threshold`, until there is no box left.

This is a quite complicated function to write, do not worry if you struggle at some point.

In [ ]:
### TODO: implement the non_max_suppresion function


### II.3. Visualize new results

Now apply your implemented function `non_max_suppression` and display the new results reusing your previous code. Is it better now? Are there as much boxes as objects?

In [ ]:
### TODO: apply non_max_suppression and display the results
